In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from langchain_huggingface import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableMap, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [4]:
model_name = "Qwen/Qwen1.5-1.8B-Chat"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

hf_pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=200, temperature=0.5, do_sample=True)

llm = HuggingFacePipeline(pipeline=hf_pipe)

Device set to use cpu


In [6]:
llm.invoke("who are you")

"who are you? i am a large language model created by Alibaba Cloud. my name is QianWen, and I am one of the most advanced AI models in the world. I was trained on a vast amount of text data from various sources, including books, articles, websites, and other online resources, to enable me to understand natural language and generate human-like responses.\nQianWen has been designed to perform a wide range of tasks, such as answering questions, providing information, generating text, translating languages, and even playing games like chess or Go. My ability to understand context, recognize patterns, and generate coherent responses is based on my deep neural network architecture, which allows me to learn and adapt to new inputs over time.\nI am constantly being updated and improved by Alibaba Cloud's team of experts, who work tirelessly to ensure that I remain at the forefront of AI technology. Whether you're looking for assistance with a specific task or want to engage in a conversation w

In [7]:
llm.invoke("what are the steps to solve a quadratic equaion")

'what are the steps to solve a quadratic equaion using substitution method? To solve a quadratic equation using the substitution method, you can follow these steps:\n\n1. Identify the quadratic equation: Write down the quadratic equation in standard form (ax^2 + bx + c = 0), where a, b, and c are constants and x is the variable.\n\n   Example:\n   - Quadratic equation: 3x^2 + 5x + 2 = 0\n\n2. Solve for one variable: Choose one of the variables, say x, and express it as a function of the other variable, say y. This will be done by setting up an equation based on the quadratic formula:\n\n   - If the coefficient of x^2 is positive, then set x = y/2:\n     - a(y/2)^2 + b(y/2) + c = 0\n     - a(y^2/4) + b(y) + c = 0\n\n   - If the coefficient of x^2'

In [9]:
prompt = PromptTemplate(
    input_variables=['question'],
    template="Explain what is {question}."
)

chain = prompt | llm

print(chain.invoke({"question":"solar energy"}))

Explain what is solar energy. Solar energy is the energy that comes from the sun and can be harnessed to generate electricity or heat for homes, businesses, and other structures. This energy is generated by photovoltaic (PV) panels, which are made of silicon cells that absorb sunlight and convert it into electrical energy.
The process of generating solar energy involves several steps:

1. Sunlight Absorption: The PV panels are placed on a rooftop or in an open area where they can receive direct sunlight throughout the day. When sunlight hits the PV panel, it excites the electrons within the silicon cells, creating a flow of electricity.

2. Conversion of Light Energy to Electrical Energy: The excited electrons move through a series of semiconductor materials called p-n junctions, which create an electric field that opposes the flow of current. This creates a voltage difference between the two sides of the cell, which can be used to power electronic devices such as lights, appliances, a

In [10]:
name_prompt = PromptTemplate(
    input_variables=['cuisine'],
    template="Suggest me a {cuisine} resturant name."
)

chain = name_prompt | llm
print(chain.invoke("Italian"))

Suggest me a Italian resturant name. Sure, here's an Italian restaurant name suggestion:
"La Bella Italia"
This name combines the words "la Bella," which means "the beautiful" or "lovely," and "Italia," which refers to Italy. The name suggests a cozy and inviting atmosphere with a focus on traditional Italian cuisine, making it a great choice for a restaurant that wants to appeal to customers who are interested in Italian food and culture.
Other potential names for an Italian restaurant could include:
- Ristorante Italiano
- Il Pasticceria di Roma
- La Trattoria del Gusto
- Osteria del Vino
- Roscioli e Fritti
- Il Buon Caffè
- La Trattoria del Gnocchi
- Pizzeria Toscana
I hope this helps! Let me know if you have any other questions or need further suggestions.


In [11]:
template = "what is {question}"
prompt = PromptTemplate.from_template(template)

chain = prompt | llm

print(chain.invoke({"question": "dangling pointer"}))

what is dangling pointer in c++
In C++, a "dangling pointer" refers to a variable that holds the address of another variable, but it does not point to the actual memory location where the variable is stored. This can happen when a pointer variable is declared and initialized with the address of another variable, but the pointer itself is not assigned to any valid memory location.

Here's an example:

```c++
int *p = &x; // p holds the address of x
```

In this example, `p` is a pointer to an integer variable `x`. The `&` operator is used to extract the address of `x` from `p`, which is then assigned to the value of `*p`. However, since `p` is not actually pointing to the memory location where `x` is stored, it is considered a dangling pointer because it points to the address of `x` rather than the actual memory location.

To fix a dangling pointer, you need to assign the correct


In [16]:
p1 = PromptTemplate.from_template("Suggest me any 1 topic name.") | llm
p2 = PromptTemplate.from_template("Give me only 1 question based on {topic}.") | llm
p3 = PromptTemplate.from_template("Give me answer to {question}") | llm

chain = RunnableMap({
    "topic":p1
    }).assign({
        "question": lambda x: p2.invoke({"topic": x["topic"]})
        }).assign({
            "answer": lambda x: p3.invoke({"question": x["question"]})
            })

print(chain.invoke({}))

TypeError: Runnable.assign() takes 1 positional argument but 2 were given

In [19]:
p1 = PromptTemplate.from_template("Suggest a topic in computer science.") | llm
p2 = PromptTemplate.from_template("Give me only 1 question based on {topic}.") | llm
p3 = PromptTemplate.from_template("Give me answer to {question}.") | llm

# Wrap lambda calls in RunnableLambda
q_chain = RunnableLambda(lambda x: {"question": p2.invoke({"topic": x["topic"]})})
a_chain = RunnableLambda(lambda x: {"answer": p3.invoke({"question": x["question"]})})

# Chain everything
chain = (
    RunnableMap({"topic": p1}) |
    q_chain |
    a_chain
)

# Run it
result = chain.invoke({})
print(result)

{'answer': 'Give me answer to Give me only 1 question based on Suggest a topic in computer science. One interesting and relevant topic in computer science is the development of artificial intelligence (AI) systems that can learn from data, make predictions or decisions, and interact with humans in natural language. This field encompasses various subfields such as machine learning, deep learning, natural language processing (NLP), computer vision, robotics, and expert systems, among others. AI systems have numerous applications across industries, including healthcare, finance, transportation, education, entertainment, and more, and are becoming increasingly sophisticated and autonomous. As technology advances, there is a growing demand for AI systems that can perform complex tasks, adapt to changing environments, and provide personalized experiences to users. Therefore, developing AI systems that can revolutionize various aspects of our lives is an exciting and challenging area of resea

In [23]:
t = "Convert this to {language}: when is the next event"
p = PromptTemplate.from_template(t)
output_parser = StrOutputParser()

c = p | llm | output_parser

output = c.invoke({"language": "french"})

In [25]:
output.strip().split('\n')

['Convert this to french: when is the next event Sure, here\'s the translation of "when is the next event" in French:',
 'Quand est la prochaine événement? ',
 '',
 'Note that "quand" means "when", "est" means "is", and "la prochaine événement" translates to "the next event". The verb "être" (to be) is conjugated in the future tense for the second person singular form ("je") and agrees with the subject pronoun "la prochaine". So the complete sentence would be "Quand est la prochaine événement? Je suis sûr que ce sera le prochain événement." which roughly translates to "When is the next event? I am sure it will be the next event."']